In [15]:
!pip install openai langchain-community langchain-chroma langchain langchain-text-splitters langchain_core langchain_huggingface pypdf

In [40]:
from openai import OpenAI
import re
from langchain_core.prompts import PromptTemplate

In [41]:
BASE_URL = "https://openrouter.ai/api/v1"
OPEN_API_KEY = "OPENROUTER_API_KEY"...........................................YOUR_API_KEY
MODEL_1 = "mistralai/devstral-2512:free"
MODEL_2 = "xiaomi/mimo-v2-flash:free"

In [42]:
client=OpenAI(
    base_url=BASE_URL,
    api_key=OPEN_API_KEY
)

In [43]:
router_prompt = """
You are an intent classification system for an AI assistant.

Your task is to analyze the user's input and select the SINGLE most appropriate route
from the available options.

Available Routes:
{keys}

How to Decide:
- Choose "code" if the user explicitly asks for:
  • code, implementation, function, class, script
  • debugging, fixing errors, optimization
  • programming languages, APIs, frameworks
- Choose "text" if the user asks for:
  • explanations, definitions, summaries
  • concepts, theory, comparisons
  • step-by-step reasoning without code
- If the input contains both explanation and code:
  • Choose the route that best matches the PRIMARY intent
- If the intent is unclear:
  • Choose the safest and most general route

Strict Rules:
- Output ONLY valid XML
- Do NOT explain your decision
- Do NOT include markdown, comments, or extra text
- The route name MUST exactly match one of the available routes

Output Format:
<selection>route_name</selection>

User Input:
{user_input}
"""


In [44]:
routes = {
    "text": """
You are an AI assistant specialized in natural language understanding.

Your tasks include:
- Explaining concepts clearly
- Answering theoretical questions
- Summarizing content
- Providing step-by-step reasoning without writing code

Guidelines:
- Be concise but thorough
- Use simple language
- Avoid unnecessary formatting
""",

    "code": """
You are an AI assistant specialized in software development.

Your tasks include:
- Writing clean and correct code
- Debugging and fixing errors
- Optimizing existing code
- Adding meaningful comments and explanations

Guidelines:
- Use best practices
- Ensure code is executable
- Do not include unnecessary text outside code explanations
"""
}


In [45]:
def prompt_making(user_input,keys, router_prompt):
    prompt = PromptTemplate(
        input_variables=["keys","user_input"],
        template=router_prompt
    )
    return prompt.format(input=user_input)

In [46]:

def response_generation(final_input, model_name):
    response_obj = client.responses.create(
        model=model_name,
        input=final_input,
        max_output_tokens=1024,
        temperature=0.7,
        extra_headers={
            "HTTP-Referer": "https://company-chatbot.local",
            "X-Title": "Company RAG Chatbot",
        }
    )

    text = response_obj.output_text

    # Safe cleaning (do NOT break code formatting)
    clean_text = re.sub(r'[##_]+', '', text)

    return clean_text.strip()


In [47]:
def extract_xml(text, tag):
    match = re.search(rf"<{tag}>(.*?)</{tag}>", text, re.DOTALL | re.IGNORECASE)
    return match.group(1).strip() if match else ""

In [50]:
def route(user_input, routes):
    print(f"\nAvailable routes: {list(routes.keys())}")

    router_prompt = f"""
You are an intent classification system for an AI assistant.

Available Routes:
{list(routes.keys())}

Decision Rules:
- Choose "code" if the user asks for programming, code, debugging, or implementation
- Choose "text" if the user asks for explanation, theory, or summaries
- Choose the primary intent if the input is mixed
- If unsure, choose the safest route

Strict Rules:
- Output ONLY valid XML
- No explanations
- Route name must exactly match one of the available routes

Format:
<selection>route_name</selection>

User Input:
{user_input}
""".strip()

    # Routing decision → MODEL_1
    route_response = response_generation(router_prompt, MODEL_1)

    route_key = extract_xml(route_response, "selection").lower()

    if route_key not in routes:
        raise ValueError(f"Invalid route selected: {route_key}")

    print(f"Selected route → {route_key}")

    # Select correct model
    model_to_use = MODEL_2 if route_key == "code" else MODEL_1

    final_prompt = f"{routes[route_key]}\n\nUser Input:\n{user_input}"

    return response_generation(final_prompt, model_to_use)


In [51]:
output = route(
    user_input="Write Python code for self-attention from scratch",
    routes=routes
)

print("\nFinal Output:\n", output)



Available routes: ['text', 'code']
Selected route → code

Final Output:
 ```python
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class SelfAttention(nn.Module):
    """
    Self-attention mechanism from scratch using PyTorch.
    
    Implements scaled dot-product attention as described in "Attention Is All You Need" (Vaswani et al., 2017).
    
    Formula:
        Attention(Q, K, V) = softmax(QK^T / sqrt(dk)) * V
    
    Args:
        embeddim (int): Total dimension of the input embeddings
        numheads (int): Number of parallel attention heads
        dropout (float, optional): Dropout probability on attention weights. Default: 0.1
        bias (bool, optional): Whether to use bias in linear projections. Default: True
    
    Shape:
        Input: (batchsize, seqlen, embeddim)
        Output: (batchsize, seqlen, embeddim)
    """
    
    def init(self, embeddim: int, numheads: int, dropout: float = 0.1, bias: bool = True):
        super().in

In [52]:
output = route(
    user_input="Tell me something about great imran khan",
    routes=routes
)

print("\nFinal Output:\n", output)


Available routes: ['text', 'code']
Selected route → text

Final Output:
 Imran Khan is a prominent figure in Pakistan, known for his contributions as a **cricket legend, philanthropist, and politician**. Here’s a brief overview:

1. **Cricket Career**:
   - One of Pakistan’s greatest cricketers, leading the national team to its first **World Cup victory in 1992**.
   - A skilled all-rounder (bowler and batsman), he played from 1971 to 1992.

2. **Philanthropy**:
   - Founded **Shaukat Khanum Memorial Cancer Hospital** (1994) in Lahore, a free cancer treatment center named after his mother.
   - Established **Namal University** (2008) to provide quality education in rural areas.

3. **Political Career**:
   - Founded **Pakistan Tehreek-e-Insaf (PTI)** in 1996, a political party advocating for justice and anti-corruption.
   - Served as **Prime Minister of Pakistan (2018–2022)** but was removed via a no-confidence vote.
   - Remains a polarizing but influential leader, with a strong you